In [1]:
import pandas as pd
import numpy as np 
import pickle
import xgboost as xgb

In [2]:
pik = pickle.load(open('../saved_datasets/15-3-19,pipeline_A,dateback=12,alphas=[0,50,100]/xtrain_xtest_ytrain_ytest.pickle','rb'))

#x_train , x_test , y_train , y_test = [[x[0]] for x in pik]
x_train , x_test , y_train , y_test = pik

del pik

In [3]:
x_train[0].head()

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,month_encoded_alpha_50,city_encoded_alpha_50,item_type_encoded_alpha_50,item_info_encoded_alpha_100,shop_id_encoded_alpha_100,item_id_encoded_alpha_100,item_category_id_encoded_alpha_100,month_encoded_alpha_100,city_encoded_alpha_100,item_type_encoded_alpha_100
0,12,2,32,40,0.0,0.0,0.0,0.0,0.0,0.0,...,0.345510,0.149538,0.207502,0.245058,0.149622,0.654339,0.245058,0.345508,0.149622,0.207506
1,12,2,33,37,42.0,1322.0,1.0,9959.0,55.0,15.0,...,0.342765,0.154742,0.207805,0.162711,0.154824,0.394627,0.162711,0.342763,0.154824,0.207809
2,12,2,99,37,0.0,0.0,0.0,0.0,0.0,0.0,...,0.353070,0.154511,0.207298,0.162172,0.154593,0.188741,0.162172,0.353067,0.154593,0.207301
3,12,2,482,73,88.0,1322.0,1.0,419.0,3.0,85.0,...,0.350848,0.153088,0.182709,0.388219,0.153170,1.320853,0.388219,0.350845,0.153170,0.182753
4,12,2,485,73,118.0,1322.0,1.0,419.0,3.0,0.0,...,0.342765,0.154742,0.182086,0.384908,0.154824,1.669637,0.384908,0.342763,0.154824,0.182131


# Preprocessing

In [4]:
def preproc_model3(x_train,x_test,y_train,y_test):
    x_train = [d.drop(['date_block_num','shop_id','item_id','item_category_id','month','city','item_type','item_info'],axis=1) for d in x_train]
    x_test =  [d.drop(['date_block_num','shop_id','item_id','item_category_id','month','city','item_type','item_info'],axis=1) for d in x_test]
    y_train = [d.clip(0,20) for d in y_train]
    y_test =  [d.clip(0,20) for d in y_test]
    return [x_train,x_test,y_train,y_test]

In [5]:
x_train , x_test , y_train , y_test = preproc_model3(x_train,x_test,y_train,y_test)

# HACKS

### sort columns

In [6]:
train_cols = x_train[0].columns.tolist()
x_test[0] = x_test[0][train_cols]

In [7]:
A = x_train[0].columns.tolist()

In [8]:
B = x_test[0].columns.tolist()

In [9]:
A==B

True

### fillnas in y_test

In [10]:
y_test = [d.fillna(0) for d in y_test]

## TODO: account for factor adjustment

# Eval Metric

In [11]:
np.array([1,2,3]).clip(0,1)

array([1, 1, 1])

In [12]:
def model3_eval(preds,true_y):
    preds_clipped = preds.clip(0,20)
    true_y_clipped = true_y.clip(0,20)
    sq_res = (preds_clipped - true_y_clipped)**2
    return  np.sqrt(np.mean(sq_res))

# Objective metric

In [13]:
#def model3_objective(preds,true_y):
    

In [14]:
np.mean(y_test[0].isna())

0.0

# Notes

- There are likely too many feats, this would explain immediate overfitting
- rmse withou clipped data is gonna put a lot of focus on large outliers
* consider mean abs error

# Feature Selection

In [15]:
#x_train[0].columns.tolist()

In [16]:
def feat_selection(x_train,x_test,y_train,y_test):
    FEATS= ['sum_item_sales_back_1',
 'sum_shop_sales_back_1',
 'item_cnt_month_back_1',
 'sum_item_cat_sales_back_1',
 'sum_item_cat_shop_sales_back_1',
 'sum_item_sales_back_2',
 'sum_shop_sales_back_2',
 'item_cnt_month_back_2',
 'sum_item_cat_sales_back_2',
 'sum_item_cat_shop_sales_back_2']#,
# 'sum_item_sales_back_3',
# 'sum_shop_sales_back_3',
# 'item_cnt_month_back_3',
# 'sum_item_cat_sales_back_3',
# 'sum_item_cat_shop_sales_back_3',
# 'sum_item_sales_back_4',
# 'sum_shop_sales_back_4',
# 'item_cnt_month_back_4',
# 'sum_item_cat_sales_back_4',
# 'sum_item_cat_shop_sales_back_4',
# 'sum_item_sales_back_5',
# 'sum_shop_sales_back_5',
# 'item_cnt_month_back_5',
# 'sum_item_cat_sales_back_5',
# 'sum_item_cat_shop_sales_back_5',
# 'sum_item_sales_back_6',
# 'sum_shop_sales_back_6',
# 'item_cnt_month_back_6',
# 'sum_item_cat_sales_back_6',
# 'sum_item_cat_shop_sales_back_6',
# 'sum_item_sales_back_7',
# 'sum_shop_sales_back_7',
# 'item_cnt_month_back_7',
# 'sum_item_cat_sales_back_7',
# 'sum_item_cat_shop_sales_back_7',
# 'sum_item_sales_back_8',
# 'sum_shop_sales_back_8',
# 'item_cnt_month_back_8',
# 'sum_item_cat_sales_back_8',
# 'sum_item_cat_shop_sales_back_8',
# 'sum_item_sales_back_9',
# 'sum_shop_sales_back_9',
# 'item_cnt_month_back_9',
# 'sum_item_cat_sales_back_9',
# 'sum_item_cat_shop_sales_back_9',
# 'sum_item_sales_back_10',
# 'sum_shop_sales_back_10',
# 'item_cnt_month_back_10',
# 'sum_item_cat_sales_back_10',
# 'sum_item_cat_shop_sales_back_10',
# 'sum_item_sales_back_11',
# 'sum_shop_sales_back_11',
# 'item_cnt_month_back_11',
# 'sum_item_cat_sales_back_11',
# 'sum_item_cat_shop_sales_back_11',
# 'sum_item_sales_back_12',
# 'sum_shop_sales_back_12',
# 'item_cnt_month_back_12',
# 'sum_item_cat_sales_back_12',
# 'sum_item_cat_shop_sales_back_12',
# 'numdays',
# 'mean_prevmonth_item_price',
# 'std_prevmonth_item_price',
# 'median_prevmonth_item_price',
# 'median_prevmonth_shop_item_price',
# 'prop_median_item_price',
# 'first_sale_date_block',
# 'new_item',
# 'item_info_encoded_alpha_0',
# 'shop_id_encoded_alpha_0',
# 'item_id_encoded_alpha_0',
# 'item_category_id_encoded_alpha_0',
# 'month_encoded_alpha_0',
# 'city_encoded_alpha_0',
# 'item_type_encoded_alpha_0',
# 'item_info_encoded_alpha_50',
# 'shop_id_encoded_alpha_50',
# 'item_id_encoded_alpha_50',
# 'item_category_id_encoded_alpha_50',
# 'month_encoded_alpha_50',
# 'city_encoded_alpha_50',
# 'item_type_encoded_alpha_50',
# 'item_info_encoded_alpha_100',
# 'shop_id_encoded_alpha_100',
# 'item_id_encoded_alpha_100',
# 'item_category_id_encoded_alpha_100',
# 'month_encoded_alpha_100',
# 'city_encoded_alpha_100',
# 'item_type_encoded_alpha_100']
    
    x_train = [d[FEATS] for d in x_train]
    x_test =  [d[FEATS] for d in x_test]
    return [x_train,x_test,y_train,y_test]

In [17]:
x_train , x_test , y_train , y_test = feat_selection(x_train,x_test,y_train,y_test)

# Reduce Dataset size

In [18]:
x_train_small = [d.sample(frac=1,random_state=0) for d in x_train]
y_train_small = [d.sample(frac=1,random_state=0) for d in y_train]
x_test_small = [d.sample(frac=1,random_state=0) for d in x_test]
y_test_small = [d.sample(frac=1,random_state=0) for d in y_test]

# Fitting

In [19]:
model = xgb.XGBRegressor(
    seed=0,
    max_depth=3,
    learning_rate=1,
    n_estimators=1000,
    objective='reg:linear',
)
eval_set = [(x_train_small[0],y_train_small[0]),(x_test[0],y_test[0])]

In [20]:
model.fit(
    verbose=True,
    X=x_train_small[0],
    y=y_train_small[0],
    eval_set=eval_set,
    early_stopping_rounds=50
)

[04:27:02] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
[0]	validation_0-rmse:0.958595	validation_1-rmse:1.26307
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:0.944925	validation_1-rmse:1.27783
[2]	validation_0-rmse:0.939023	validation_1-rmse:1.27911
[3]	validation_0-rmse:0.936721	validation_1-rmse:1.28012
[4]	validation_0-rmse:0.93411	validation_1-rmse:1.27822
[5]	validation_0-rmse:0.931692	validation_1-rmse:1.29008
[6]	validation_0-rmse:0.930155	validation_1-rmse:1.30059
[7]	validation_0-rmse:0.928799	validation_1-rmse:1.29756
[8]	validation_0-rmse:0.927448	validation_1-rmse:1.30022
[9]	validation_0-rmse:0.925664	validation_1-rmse:1.2974
[10]	validation_0-rmse:0.924589	validation_1-rmse:1.29328
[11]	validation_0-rmse:0.923275	validation_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=1)